# Example for a heat pump with a standard cycle


Let's start the complete cycle simulation with the
most basic flowsheet, the standard-cycle. As all flowsheets
contain a condenser and an evaporator, we defined a common BaseCycle
to avoid code-repetition.
We can import this flowsheet and see how to use it. Note that
modern coding IDEs like PyCharm will tell you which arguments belong
to a class or function. If you don't have that at hand, you need
to look into the documentation.


In [ ]:
from vclibpy.flowsheets import BaseCycle, StandardCycle
help(BaseCycle)
help(StandardCycle)


We fist need to define the components in the cycle.
Here we are using the components developed in the previous examples.
Also, note again that the expansion valve model does not influence the results
for the current algorithm. But, you could size the expansion valve
using vclibpy, including off-design, but this is one for another example.


In [ ]:
from vclibpy.components.heat_exchangers import moving_boundary_ntu
from vclibpy.components.heat_exchangers import heat_transfer
condenser = moving_boundary_ntu.MovingBoundaryNTUCondenser(
    A=5,
    secondary_medium="water",
    flow_type="counter",
    ratio_outer_to_inner_area=1,
    two_phase_heat_transfer=heat_transfer.constant.ConstantTwoPhaseHeatTransfer(alpha=5000),
    gas_heat_transfer=heat_transfer.constant.ConstantHeatTransfer(alpha=5000),
    wall_heat_transfer=heat_transfer.wall.WallTransfer(lambda_=236, thickness=2e-3),
    liquid_heat_transfer=heat_transfer.constant.ConstantHeatTransfer(alpha=5000),
    secondary_heat_transfer=heat_transfer.constant.ConstantHeatTransfer(alpha=5000)
)
evaporator = moving_boundary_ntu.MovingBoundaryNTUEvaporator(
    A=15,
    secondary_medium="air",
    flow_type="counter",
    ratio_outer_to_inner_area=10,
    two_phase_heat_transfer=heat_transfer.constant.ConstantTwoPhaseHeatTransfer(alpha=1000),
    gas_heat_transfer=heat_transfer.constant.ConstantHeatTransfer(alpha=1000),
    wall_heat_transfer=heat_transfer.wall.WallTransfer(lambda_=236, thickness=2e-3),
    liquid_heat_transfer=heat_transfer.constant.ConstantHeatTransfer(alpha=5000),
    secondary_heat_transfer=heat_transfer.constant.ConstantHeatTransfer(alpha=25)
)
from vclibpy.components.expansion_valves import Bernoulli
expansion_valve = Bernoulli(A=0.1)

from vclibpy.components.compressors import RotaryCompressor
compressor = RotaryCompressor(
    N_max=125,
    V_h=19e-6
)


Now, we can plug everything into the flowsheet:


In [ ]:
heat_pump = StandardCycle(
    evaporator=evaporator,
    condenser=condenser,
    fluid="Propane",
    compressor=compressor,
    expansion_valve=expansion_valve,
)


As in the other example, we can specify save-paths,
solver settings and inputs to vary:
Note that T_con can either be inlet or outlet, depending on the setting
of `use_condenser_inlet`. Per default, we simulate the inlet, T_con_in


In [ ]:
save_path = r"D:\00_temp\simple_heat_pump"
T_eva_in_ar = [-10 + 273.15, 273.15, 10 + 273.15]
T_con_ar = [30 + 273.15, 50 + 273.15, 70 + 273.15]
n_ar = [0.3, 0.7, 1]


Now, we can generate the full-factorial performance map
using all inputs. The results will be stored under the
save-path. To see some logs, we can import the logging module
and get, for example, all messages equal or above the INFO-level


In [ ]:
import logging
logging.basicConfig(level="INFO")

from vclibpy import utils
save_path_sdf, save_path_csv = utils.full_factorial_map_generation(
    heat_pump=heat_pump,
    save_path=save_path,
    T_con_ar=T_con_ar,
    T_eva_in_ar=T_eva_in_ar,
    n_ar=n_ar,
    use_condenser_inlet=use_condenser_inlet,
    use_multiprocessing=False,
    save_plots=True,
    m_flow_con=0.2,
    m_flow_eva=0.9,
    dT_eva_superheating=5,
    dT_con_subcooling=0,
)


What did just happen? We can analyze all results by listing the
files in the save-path - or just open it in our default system explorer.


In [ ]:
import os
print(os.listdir(save_path))


One file should be: `Standard_Propane.csv`. We can load this file and plot


In [ ]:
import pandas as pd
df = pd.read_csv(save_path_csv, index_col=0)
df


Now, we can plot variables, for example as a scatter plot using matplotlib.
You have to know the names, which are the column headers.


In [ ]:
import matplotlib.pyplot as plt
x_name = "n in - (Relative compressor speed)"
y_name = "COP in - (Coefficient of performance)"
plt.scatter(df[x_name], df[y_name], s=20)
plt.ylabel(y_name)
plt.xlabel(x_name)
plt.show()


Looking at the results, we see that a higher frequency often leads to lower COP values.
However, other inputs (temperatures) have a greater impact on the COP.
We can also use existing 3D-plotting scripts in vclibpy to analyze the
dependencies. For this, we need the .sdf file. In the sdf, the field names are without
the unit and description, as those are accessible in the file-format in other columns.
Depending on whether we varied the inlet or outlet, we have to specify the correct name.


In [ ]:
from vclibpy.utils.plotting import plot_sdf_map
plot_sdf_map(
    filepath_sdf=save_path_sdf,
    nd_data="COP",
    first_dimension="T_eva_in",
    second_dimension="T_con_in" if use_condenser_inlet else "T_con_out",
    fluids=["Propane"],
    flowsheets=["Standard"]
)
